In [1]:
import pandas as pd
import numpy as np

In [2]:
df_pol_1 = pd.read_csv('./政治/New_New_ControlofCorruption.csv').sort_values(by='Country Name')
df_pol_2 = pd.read_csv('./政治/New_New_PoliticalStability.csv').sort_values(by='Country Name')
df_pol_3 = pd.read_csv('./政治/New_New_GovernmentEffectiveness.csv').sort_values(by='Country Name')
df_pol_4 = pd.read_csv('./政治/New_New_RegulatoryQuality.csv').sort_values(by='Country Name')
df_pol_5 = pd.read_csv('./政治/New_New_RuleofLaw.csv').sort_values(by='Country Name')
df_pol_6 = pd.read_csv('./政治/New_New_VoiceandAccountability.csv').sort_values(by='Country Name')

df_eco_1 = pd.read_csv('./经济/New_New_GDP per capita.csv').sort_values(by='Country Name')
df_eco_2 = pd.read_csv('./经济/New_New_GDP growth.csv').sort_values(by='Country Name')
df_eco_3 = pd.read_csv('./经济/New_New_debt index.csv').sort_values(by='Country Name')
df_eco_4 = pd.read_csv('./经济/New_New_Inflation rate.csv').sort_values(by='Country Name')
df_eco_5 = pd.read_csv('./经济/New_New_fiscal balance index.csv').sort_values(by='Country Name')
df_eco_6 = pd.read_csv('./经济/New_New_trade index.csv').sort_values(by='Country Name')

df_soc_1 = pd.read_csv('./社会/New_New_unemployment.csv').sort_values(by='Country Name')
df_soc_2 = pd.read_csv('./社会/New_New_School enrollment.csv').sort_values(by='Country Name')
df_soc_3 = pd.read_csv('./社会/New_New_Life expectancy.csv').sort_values(by='Country Name')
df_soc_4 = pd.read_csv('./社会/New_New_Intentional homicides.csv').sort_values(by='Country Name')
df_soc_5 = pd.read_csv('./社会/New_New_gini index.csv').sort_values(by='Country Name')

average_df = pd.DataFrame()
index_weight = pd.read_excel('./index.xlsx')
labelind = pd.read_csv('./score_sort.csv').keys()[1:-2]
average_df[labelind[0]] = df_pol_1[labelind[0]]

In [3]:
for i in range(2002, 2021):
    # data = f'{year}'
    df = pd.DataFrame()
    labelind = pd.read_csv('./score_sort.csv').keys()[1:-2]
    df[labelind[0]] = df_pol_1[labelind[0]]
    labeldate = df_pol_1.keys()[2:-1]
    y = i - 2002
    df[labelind[1]] = df_pol_1[labeldate[y]].values
    df[labelind[2]] = df_pol_2[labeldate[y]].values
    df[labelind[3]] = df_pol_3[labeldate[y]].values
    df[labelind[4]] = df_pol_4[labeldate[y]].values
    df[labelind[5]] = df_pol_5[labeldate[y]].values
    df[labelind[6]] = df_pol_6[labeldate[y]].values

    df[labelind[7]] = df_eco_1[labeldate[y]].values
    df[labelind[8]] = df_eco_2[labeldate[y]].values
    df[labelind[9]] = df_eco_3[labeldate[y]].values
    df[labelind[10]] = df_eco_4[labeldate[y]].values
    df[labelind[11]] = df_eco_5[labeldate[y]].values
    df[labelind[12]] = df_eco_6[labeldate[y]].values

    df[labelind[13]] = df_soc_1[labeldate[y]].values
    df[labelind[14]] = df_soc_2[labeldate[y]].values
    df[labelind[15]] = df_soc_3[labeldate[y]].values
    df[labelind[16]] = df_soc_4[labeldate[y]].values
    df[labelind[17]] = df_soc_5[labeldate[y]].values


    def minToMax(datas):
        return datas.max() - datas
    df['debt index'] = minToMax(df['debt index'])
    df['Intentional homicides'] = minToMax(df['Intentional homicides'])

    def interToMax(datas, a, b):
        M = max(a - datas.min(), datas.max() - b)
        ans = []
        for data in datas:
            if data < a:
                ans.append(1 - (a - data) / M)
            elif data > b:
                ans.append(1 - (data - b) / M)
            else:
                ans.append(1)
        return ans

    df['Inflation rate'] = interToMax(df['Inflation rate'], 3, 5)
    df['unemployment'] = interToMax(df['unemployment'], 4, 8)
    df['gini index'] = interToMax(df['gini index'], 30, 40)

    def standard(datas):
        k = np.power(np.sum(np.power(datas, 2), axis=0), 0.5)
        for i in range(len(k)):
            datas[:, i] = datas[:, i] / k[i]
        return datas
    label = df.keys()[1:]
    data = standard(df[label].values)
    df[label] = data
    
    label = df.keys()[1:]
# 负值正向化
    a = np.array(df[label])
    a = a + abs(a.min(axis=0)) + 0.01
# 熵权法得到权重w
    [n, m] = a.shape
    cs = a.sum(axis=0)
    P = a / cs
    e = -(P * np.log(P)).sum(axis=0) / np.log(n)
    g = 1 - e
    w = g / sum(g)
# TOPSIS评分
    def score(datas):
        z_min = np.min(datas, axis = 0)
        z_max = np.max(datas, axis = 0)
        min_Distance = np.power(np.power(z_min - datas, 2) @ w, 0.5)
        max_Distance = np.power(np.power(z_max - datas, 2) @ w, 0.5)
        scores = min_Distance / (max_Distance + min_Distance)
        # scores = scores / np.sum(scores, axis = 0)
        return scores
    average_df[f'{y + 2002}'] = score(a)
    index_weight[f'{y + 2002}'] = w

In [5]:
index_weight.to_csv('index.csv')

In [11]:
average_df.to_csv('average_soc.csv')

In [11]:
df.keys()[7:13]

Index(['GDP per capita', 'GDP growth', 'debt index', 'Inflation rate',
       'fiscal balance', 'trade index'],
      dtype='object')

In [6]:
df.keys()[13:]

Index(['unemployment', 'School enrollment', 'Life expectancy',
       'Intentional homicides', 'gini index'],
      dtype='object')

In [7]:
hh = pd.read_excel('./121个国家18年的政治、经济、社会稳定平均值.xlsx')
hh.describe()

,pol_average,eco_average,soc_average,Score,Unnamed: 5,Unnamed: 6
count,121.000000,121.000000,121.000000,121.000000,0.0,3.000000
mean,0.480152,0.237335,0.422423,0.372652,NaN,0.333333
std,0.220008,0.113414,0.184054,0.155390,NaN,0.148154
min,0.153205,0.143669,0.137913,0.164533,NaN,0.163082
25%,0.309723,0.179107,0.254787,0.263909,NaN,0.283516
50%,0.407915,0.193194,0.393817,0.313563,NaN,0.403950
75%,0.639021,0.227415,0.556107,0.463034,NaN,0.418459
max,0.966799,0.773965,0.884761,0.803642,NaN,0.432968
